In [ ]:
import math
import matplotlib.pyplot as plt
import numpy as np
import random
import sys
print(sys.getrecursionlimit())
from scipy.integrate import solve_ivp
sys.setrecursionlimit(5000)

In [ ]:
def ColMax(x,i):
    mx =0;
    for j in range (len(x)):
        mx = max(mx, x[j][i])
    return mx

def Beam(t,x,k0,k2,k3,Cp,neta,Rl,w,A,m,c):
    dxdt=np.zeros(len(x))
    dxdt[0]=x[1];
    dxdt[1]=(1/m)*(-neta*A*math.sin(w*t) - c*x[1] -k2*x[0] -k3*(x[0]**3)- k0*x[2]);
    dxdt[2]=(1/Cp)*(-(x[2]/Rl)+ k0*x[1]);
    return dxdt

# Unoptimized Non-linear Beam

In [ ]:
E=np.array([170e9,300e9]);
t1 = 50e-6;
t2 = 1e-6;
T=np.array([t1,t2]);
density=np.array([2330,3320]);
n = 2;#E.size();
length = 30e-3;
width = 5e-3;
lp = 10e-3;
Area=T*width;
volume=width*length*T;
mass=sum(volume*density);
m_added=5e-4; # mass added to reduce the resonant frequency
meff=0.37*mass+m_added;
e0=8.85e-12;
er=8.8;
Cp=width*lp*e0*er/t2; # capacitance of piezo

S=np.zeros(n);
for i in range(n):
    S[i]=sum(T[0:i+1])-T[i]/2;
N=np.array(E/E[0]);

Neu3=sum(Area*N*S)/sum(Area*N); #% neutral_axis_location from bottom

#%calculation of Hu,Hl
HL=np.zeros(n);
for i in range(n):
    HL[i]=(sum(T[0:i+1])-Neu3);
HU=np.array(HL-T);

KL=(2*(3.14**4)*width*sum(E*abs(HU**3-HL**3))/(3*(length**3))); # linear stifness of the beam
KN=abs((3.14**4)*width*sum(E*(HU - HL))/(8*(length**3))); # effective nonlienar stiffness of the beam

wn=(abs(KL)/meff)**0.5;
fn=wn/(2*3.14);
Q=20;
c=wn*meff/Q;
em_couple=0.01; # K33_square
k0=(em_couple*KL*Cp)**0.5; # em coupling coefficient
Rl=1/(2*3.14*Cp*fn); # load resistance

tspan=np.linspace(0,2,2500); # time span of simulation
x0=[0,0,0];
A=5;

In [ ]:
f=np.linspace(50,250,101);
w1=2*3.14*f;

volt = np.zeros(len(w1));
disp = np.zeros(len(w1));

for i in range(len(w1)):
    sol=solve_ivp(lambda T,X: Beam(T,X,k0,KL,KN,Cp,meff,Rl,w1[i],A,meff,c),[0,2],x0,rtol=1e-6,t_eval=tspan);
    t=np.array(sol.t);
    x=np.array(sol.y);
    volt[i] = max(x[2,int(0.7*len(x[2])):len(x[2])]);
    disp[i] = max(x[0,int(0.7*len(x[0])):len(x[0])]);
    n0 = len(x[0]); 
    x0 = x[:,n0-1];
    print("iteration",i,"completed");

In [ ]:
print("Maximum Voltage: ",max(volt));

In [ ]:
plt.plot(f,disp/1e-6, color='g');
plt.xlabel('Frequency (Hz)');
plt.ylabel('Displacement (um)');
plt.show();

In [ ]:
plt.plot(f,volt, color='r');
plt.xlabel('Frequency (Hz)');
plt.ylabel('Voltage (volts)');
plt.show();

# Genetic Algorithm Optimization of Non-linear Beam

In [ ]:
def cantilever_opt(t1,t2,width,length,lp,k0):
    E=np.array([170e9,300e9]);
    T=np.array([t1,t2]);
    density=np.array([2330,3320]);
    n = 2 ;#E.size();
    Area=T*width;
    volume=width*length*T;
    mass=sum(volume*density);
    m_added=5e-4; # mass added to reduce the resonant frequency
    meff=0.37*mass+m_added;
    e0=8.85e-12;
    er=8.8;
    Cp=width*lp*e0*er/t2; # capacitance

    S=np.zeros(n);
    for i in range(n):
        S[i]=sum(T[0:i+1])-T[i]/2;
    N=np.array(E/E[0]);

    Neu3=sum(Area*N*S)/sum(Area*N); # neutral_axis_location from bottom

    #%calculation of Hu,Hl
    HL=np.zeros(n);
    for i in range(n):
        HL[i]=(sum(T[0:i+1])-Neu3);
    HU=np.array(HL-T);

    KL=(2*(3.14**4)*width*sum(E*abs(HU**3-HL**3))/(3*(length**3))); # linear stifness
    KN=abs((3.14**4)*width*sum(E*(HU - HL))/(8*(length**3))); # effective nonlienar stiffness of the beam
    wn=(abs(KL)/meff)**0.5;
    fn=wn/(2*3.14);
    Q=20;
    c=wn*meff/Q;
    em_couple=0.01; # K33_square
    k0=(em_couple*KL*Cp)**0.5; # em coupling coefficient
    Rl=1/(2*3.14*Cp*fn); # load ressistance

    tspan=np.linspace(0,2,2500); # time span of simulation
    x0=[0,0,0];
    A=5;

    f=np.linspace(50,250,101);
    w1=2*3.14*f;

    volt = np.zeros(len(w1));
    disp = np.zeros(len(w1));

    for i in range(len(w1)):
        sol=solve_ivp(lambda T,X: Beam(T,X,k0,KL,KN, Cp,meff,Rl,w1[i],A,meff,c),[0,2],x0,rtol=1e-6,t_eval=tspan);
        t=np.array(sol.t);
        x=np.array(sol.y);
        volt[i] = max(x[2,int(0.7*len(x[2])):len(x[2])]);
        disp[i] = max(x[0,int(0.7*len(x[0])):len(x[0])]);
        n0 = len(x[0]);
        x0 = x[:,n0-1];
        print("iteration",i,"completed");
    print("Maximum voltage:", max(volt));
    print("--------------------------------------");
    return max(volt);

In [ ]:
def Compare_Fitness(a,b):
    if fitness(a)>fitness(b):
        return True
    else:
        return False

def mutate_N(V1):
    a2= random.randint(1, len(V1)-1)     
    for i in range(a2): 
        a1 = random.randint(0, len(V1)-1) 
        sample = V1[a1]*(10**6)
        mu, sigma = 0, sample/4  
        s = int(np.random.normal(mu, sigma, 1)) 
        V1[a1]= (abs(V1[a1]+s))/(10**6) 
        if(V1[a1]> S[a1][1]): 
            V1[a1]= S[a1][1] 
        elif(V1[a1] < S[a1][0]):
            V1[a1] = S[a1][0]
    return V1

def Evolve(M,prev_peak, err , peak_list,AVG,MED):
    POPULATION_SIZE = len(M) 
    M=sorted(M,key=lambda x: fitness(x)) 
    peak = fitness(M[-1]) 
    Y_lis=list(map(lambda x: fitness(x),M)) 
    avg= sum(Y_lis)/len(Y_lis) 
    AVG.append(avg) 
    MED.append(Y_lis[int(len(Y_lis)/2)]) 
    peak_list.append(peak) 
    if(len(peak_list)> 20 and (max(peak_list[-20:])-min(peak_list[-20:])) < err):
       #print("I am printing ", M[-1]) 
        return M[-1]  
    E = Elites(M) 
    s = int((90*POPULATION_SIZE)/100) 
    M=M[int(len(M)/2):] 
    
    R2_Set=[[0 for k1 in range(2)] for k2 in range(len(M)**2 - len(M))]     
    indices = [0] *(len(M)**2 - len(M)) 
    count = 0; 
    for i in range(len(M)): 
        for j in range(len(M)):
            if i==j:
                continue 
            else: 
                R2_Set[count] = [i,j] 
                indices[count] = count 
                count+=1 
    P1= np.random.choice(indices, s , replace=False)     
    for p1 in range(len(P1)): 
        #print(list(map(lambda x:fitness(x),E))) 
        Parent1_i = R2_Set[P1[p1]][0] 
        Parent2_i = R2_Set[P1[p1]][1] 
        Parent1 = M[Parent1_i] 
        Parent2 = M[Parent2_i] 
        Child = Mate(Parent1, Parent2) 
        E.append(Child)
    
    samp_mut_size=int(len(E)*0.5) 
    samp_mut=random.sample(range (0,len(E)),samp_mut_size)     
    for i in samp_mut: 
        E[i]=mutate_N(E[i])
    
    return Evolve(E,peak,err,peak_list,AVG,MED)
    
# for the purpose of Elitism 
def Mate(E1, E2): 
    half_len = int(len(E1)/2) 
    a = random.randint(0,half_len)     
    b = a+half_len 
    ret1 = E1[:a]+E2[a:b]+E1[b:]     
    return ret1 

def Elites(M):
    j = int(0.1*len(M)) 
    if (j==0): 
        j=1 
    A=M[len(M)-j:] 
    #print(list(map(lambda x:fitness(x),A)))     
    for i in range(len(A)): 
        V1= A[i].copy()  
        V2= mutate_N(V1) 
        if(fitness(V2)>fitness(A[i])):
            A[i]=V2
    return A  
    
def fitness(V): 
    return cantilever_opt(V[0],V[1],V[2],V[3],V[4],-1*V[5]); 

## creating a population matrix 
def Random_Pop(S,init_size): 
    M=[[0 for j in range(len(S))]for i in range(init_size)]     
    for i in range(init_size):
        for j in range(len(S)):
            M[i][j] = random.randint(int(10**8*S[j][0]),int(10**8*S[j][1]))/10**8     
    return M 

def Elevated_Pop(S,init_size): 
    M=[] 
    S1= np.array([50e-6,1e-6,5e-3,25e-3,5e-3,1e-4])     
    S2= np.array([75e-6,2e-6,10e-3,50e-3,20e-3,1e-2]) 
    elev = S2-S1 
    factor = 1/(init_size) 
    ct = 0.05 
    while(ct<1):
        mi = S1 + (ct*elev) 
        mi = mi.tolist() 
        #print(mi) 
        M.append(mi)
        ct+= factor
    return M

In [ ]:
S =[[50e-6,75e-6],[1e-6, 2e-6],[5e-3, 10e-3],[25e-3,50e-3],[5e-3,20e-3],[1e-4,1e-2]]; 
init_size = 10; 

# M = Random_Pop(S,init_size) 
M = Elevated_Pop(S,init_size);
print("This is the initial population: ",M) 

peakL=[] 
AVG=[] 
MED=[] 
to_tes=Evolve(M,0,10**-14,peakL,AVG,MED) 
print("Final values of the parameters are : ") 
for i in to_tes: 
    print((10**6)*i) 
print("Corresponding voltage is ",peakL[-1]) 
x = np.arange(0, len(peakL)) 
for i in range(len(to_tes)):
    fit_table=[] 
    index=[] 
    spec = to_tes[i] 
    #print("for parameter no. ",i)     
    ct=0 
    step= (S[i][1]-S[i][0])*(10**-2)     
    factor=10 
    for k1 in range(-factor,factor):
        j=(k1*step)+spec 
        if(j> S[i][1] or j< S[i][0]): 
            continue 
        temp_to_tes=to_tes.copy() 
        temp_to_tes[i]=j 
        #print(temp_to_tes,j) 
        index.append(j) 
        fit_table.append(fitness(temp_to_tes))
        
    plt.title("Parameter "+str(i)+" neighbourhood") 
    plt.xlabel("Differing values of parameter = "+str(i))     
    plt.ylabel("Voltage (volts)") 
    plt.plot( index,fit_table , color ="blue") 
    plt.show()

In [ ]:
# plotting 
plt.title("Voltage vs iterations Graph") 
plt.xlabel("No. of iterations ") 
plt.ylabel("Voltage (volts)") 
plt.plot(x, peakL,color='r',label='peak')
plt.plot(x,AVG,color='g', label='avg')
plt.plot(x,MED,color='b', label='med')
plt.legend()
plt.show()

# Optimized Non-Linear Beam

In [ ]:
E=np.array([170e9,300e9]);
t1 = 61.25e-6;
t2 = 1.45e-6;
T=np.array([t1,t2]);
density=np.array([2330,3320]);
n = 2;# E.size();
length = 33.75e-3;
width = 5e-3;
lp = 5e-3;
Area=T*width;
volume=width*length*T;
mass=sum(volume*density);
m_added=5e-4; # mass added to reduce the resonant frequency
meff=0.37*mass+m_added;
e0=8.85e-12;
er=8.8;
Cp=width*lp*e0*er/t2; # capacitance

S=np.zeros(n);
for i in range(n):
    S[i]=sum(T[0:i+1])-T[i]/2;
N=np.array(E/E[0]);

Neu3=sum(Area*N*S)/sum(Area*N); # neutral_axis_location from bottom

#%calculation of Hu,Hl
HL=np.zeros(n);
for i in range(n):
    HL[i]=(sum(T[0:i+1])-Neu3);
HU=np.array(HL-T);

KL=(2*(3.14**4)*width*sum(E*abs(HU**3-HL**3))/(3*(length**3))); # linear stifness of the beam
KN=abs((3.14**4)*width*sum(E*(HU - HL))/(8*(length**3))); # effective nonlienar stiffness of the beam
wn=(abs(KL)/meff)**0.5;
fn=wn/(2*3.14);
Q=20;
c=wn*meff/Q;
em_couple=0.01; # K33_square
k0=(em_couple*KL*Cp)**0.5; # em coupling coefficient
Rl=1/(2*3.14*Cp*fn); # load resistance

tspan=np.linspace(0,2,2500); # time span of simulation
x0=[0,0,0];
A=5;

In [ ]:
f=np.linspace(50,250,101);
w1=2*3.14*f;

volt = np.zeros(len(w1));
disp = np.zeros(len(w1));

for i in range(len(w1)):
    sol=solve_ivp(lambda T,X: Beam(T,X,k0,KL,KN,Cp,meff,Rl,w1[i],A,meff,c),[0,2],x0,rtol=1e-6,t_eval=tspan);
    t=np.array(sol.t);
    x=np.array(sol.y);
    volt[i] = max(x[2,int(0.7*len(x[2])):len(x[2])]);
    disp[i] = max(x[0,int(0.7*len(x[0])):len(x[0])]);
    n0 = len(x[0]); 
    x0 = x[:,n0-1];
    print("iteration",i,"completed");

In [ ]:
print("Maximum Voltage: ",max(volt));

In [ ]:
plt.plot(f,disp/1e-6, color='g');
plt.xlabel('Frequency (Hz)');
plt.ylabel('Displacement (um)');
plt.show();

In [ ]:
plt.plot(f,volt, color='r');
plt.xlabel('Frequency (Hz)');
plt.ylabel('Voltage (volts)');
plt.show();

# Power integral optimization wrt load resistance

In [ ]:
from sklearn import metrics

In [ ]:
E=np.array([170e9,300e9]);
t1 = 61.25e-6;
t2 = 1.45e-6;
T=np.array([t1,t2]);
density=np.array([2330,3320]);
n = 2; # E.size();
length = 33.75e-3;
width = 5e-3;
lp = 5e-3;
Area=T*width;
volume=width*length*T;
mass=sum(volume*density);
m_added=5e-4; #%mass added to reduce the resonant frequency
meff=0.37*mass+m_added;
e0=8.85e-12;
er=8.8;
Cp=width*lp*e0*er/t2; #%capacitance

S=np.zeros(n);
for i in range(n):
    S[i]=sum(T[0:i+1])-T[i]/2;
N=np.array(E/E[0]);

Neu3=sum(Area*N*S)/sum(Area*N); #% newtral_axis_location from bottom

#%calculation of Hu,Hl
HL=np.zeros(n);
for i in range(n):
    HL[i]=(sum(T[0:i+1])-Neu3);
HU=np.array(HL-T);

KL=(2*(3.14**4)*width*sum(E*abs(HU**3-HL**3))/(3*(length**3))); #%linear stifness of the beam
KN=abs((3.14**4)*width*sum(E*(HU - HL))/(8*(length**3))); #% effective nonlienar stiffness of the beam
wn=(abs(KL)/meff)**0.5;
fn=wn/(2*3.14);
Q=20;
c=wn*meff/Q;
em_couple=0.01; #K33_square
k0=(em_couple*KL*Cp)**0.5; #em coupling ratio

#Rl=1/(2*3.14*Cp*fn); # load resistance
R = np.linspace(1e5,40e5,20);

tspan=np.linspace(0,2,2500); #%time span of simulation
x0=[0,0,0];
A=5;

f=np.linspace(50,250,101);
w1=2*3.14*f;

volt = np.zeros([len(R),len(w1)]);
disp = np.zeros([len(R),len(w1)]);

area_int  = np.zeros(len(R));
max_volt  = np.zeros(len(R));

for j in range(len(R)):
    print(j,"-------------------");
    for i in range(len(w1)):
        sol=solve_ivp(lambda T,X: Beam(T,X,k0,KL,KN,Cp,meff,R[j],w1[i],A,meff,c),[0,2],x0,rtol=1e-6,t_eval=tspan);
        t=np.array(sol.t);
        x=np.array(sol.y);
        volt[j,i] = max(x[2,int(0.7*len(x[2])):len(x[2])]);
        disp[j,i] = max(x[0,int(0.7*len(x[0])):len(x[0])]);
        n0 = len(x[0]);
        x0 = x[:,n0-1];
        print("iteration",i,"completed");
    max_volt[j]=max(volt[j,:]);
    area_int[j]= metrics.auc(f,(volt[j,:]*volt[j,:])/R[j]);

In [ ]:
print(area_int)

In [ ]:
plt.plot(R, area_int*1e3, color='m');
plt.xlabel('Load resistance (ohm)');
plt.ylabel('Power integral (mW-Hz)');
plt.show();

In [ ]:
plt.plot(R, max_volt, color='m');
plt.xlabel('Load resistance (ohm)');
plt.ylabel('Maximum Voltage (volts)');
plt.show();